# Chapter 2 - End-to-End ML Project

## Links
* [Numpy Ref](https://docs.scipy.org/doc/numpy/reference/)

## Hello

In [1]:
print("Hello World!")

Hello World!


## Basic Checks

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

ModuleNotFoundError: No module named 'sklearn'

## Download

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

## Quick Look

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.describe()

## Histograms

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15));
# plt.show()

## Test Set

### Home-made

In [ ]:
import numpy as np

def split_train_test(data, test_ratio, repeatable=True):
    if repeatable:
        np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices], train_indices[:3]

In [ ]:
train_set, test_set, first_three = split_train_test(housing, 0.2, repeatable=True)
print(len(train_set), "train +", len(test_set), "test. First three training indices:", first_three)

### Numpy's train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
print(len(train_set), "train +", len(test_set), "test")

### Handling test data evolution (and not messing up the test set)

* If you have an id column you could you numpy's `digest` function to hash the id using a supplied hash algorithm (e.g. `hashlib.md5`), keep the last byte, and then use only if that last byte is <= 51 (52/256 is around 20.3%)
* If there's no id, you can use data frame's `rest_index()` method to add an `index` column
  * but that only works if new data is _appended_ to the old data

### Stratified sampling

In [ ]:
# break up incomes into 5 categories:
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] <= 5, 5.0, inplace=True)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# the following is a little confusing
# 1. the for loop isn't really a for loop
# 2. it just a way of yielding from the `split.split` generator
# 3. so inside the for block, you just create the needed data frames, `strat_train_set` & `strat_test_set`
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) # get the split generator factory
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
strat_train_set["income_cat"].value_counts() / len(strat_train_set)

In [ ]:
housing["income_cat"].hist(bins=25)

In [ ]:
strat_test_set["income_cat"].hist(bins=25)

In [ ]:
strat_train_set["income_cat"].hist(bins=25)

In [ ]:
# remove the income_cat now that we've stratified succesfully:
for set_ in (strat_test_set, strat_train_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
strat_test_set.head()

## Discover and Visualize the Data to Gain Insights

### Make a copy of the training data and set aside the testing data aside (still saved as strat_test_set)

In [ ]:
housing = strat_train_set
housing.head()

### Geographical Scatterplots

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, 
            s=housing["population"]/100, label="population", figsize=(10,7),
            c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)

### Correlations

In [ ]:
corr_matrix = housing.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

### Scatter Matrix Plot

In [ ]:
# from pandas.tools.plotting import scatter_matrix
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age", "latitude"]

scatter_matrix(housing[attributes], figsize=(15,10))

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.05)

### New Attributes

In [ ]:
housing_copy = housing.copy()
housing_copy["rooms_per_household"] = housing_copy["total_rooms"]/housing_copy["households"]
housing_copy["bedrooms_per_room"] = housing_copy["total_bedrooms"]/housing_copy["total_rooms"]
housing_copy["population_per_household"] = housing_copy["population"]/housing_copy["households"]

In [ ]:
corr_matrix = housing_copy.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

## Prep the Data

In [ ]:
# X: (for now, later will become this + imputed missing values)
housing = strat_train_set.drop("median_house_value", axis=1) # drop actually makes a copy

# Y:
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
housing.head()

### Clean

#### Cleaning Options
* Get rid of rows which are missing a particular column value with `dropna`
* Get rid of an entire columne with `drop("ATTR_NAME", axis=1)`
* Default to a particular value with `fillna(THE_VALUE, inplace=True)`

### Use sklearn Imputer to Impute numerical values using medians:

In [ ]:
#DEPRECATED: from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
# must get rid of non-numericals while imputing:
housing_num = housing.drop("ocean_proximity", axis=1)

In [3]:
imputer.fit(housing_num) # doesn't modify housing_num, just computes what's necessary for imputing

NameError: name 'imputer' is not defined

In [ ]:
imputer.statistics_ # this is what will be imputed

In [ ]:
housing_num.median().values # which we can tell is just the medians (as we expected)

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
X.shape

In [ ]:
type(X), type(housing_num)

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
type(housing_tr)

### Categorical Attributes

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

#### Encode these using "one-hot encoding"

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

#### But you can make it dense as well (either call `to_array` or just initialize with dense option):

In [ ]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

### Custom Transformer

Thanks to *Scikit-Learn*'s duck typing we can build our own encoder. Just have to implement the following:
* `fit()`
* `transform()`
* `fit_transform()`

If we inherit from `BaseEstimator` we get `set_params()` and `get_params()` which are used in hyperparam tuning

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# THIS IS VERY FRAGILE!!!!
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs needed
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self # nothing else to do
        
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix]/X[:, household_ix]
        population_per_household = X[:, population_ix]/X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix]/X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
housing.shape, housing_extra_attribs.shape

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

### Feature Scaling
2 popular methods:

1. _min-max_ (aka _normalization_): shift and rescale between 0 and 1
  * cons: prone to outliers
1. _standardization_: [subract the mean and divide by the standard](https://en.wikipedia.org/wiki/Feature_scaling#Standardization_(Z-score_Normalization) deviation (Geron goofed when he said divide by the variance)
  * pros: robust against outliers
  * cons: values not bound to a range which is a problem for some algos (e.g. neural nets)

In [ ]:
### Transformation Pipelines

In [ ]:
#from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

In [ ]:
housing_num_tr = num_pipeline.fit_transform(housing_num)

housing_num_tr

#### The full pipeline

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape, housing_labels.shape

## Select and Train a Model

### Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

#### Full pipeline on a small subset

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

#### Measure the error

In [ ]:
from sklearn.metrics import mean_squared_error #l_2

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.metrics import mean_absolute_error #l_1

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
lin_mae

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)

tree_rmse = mean_squared_error(housing_labels, housing_predictions)
tree_mae = mean_absolute_error(housing_labels, housing_predictions)
tree_rmse, tree_mae

the above shows that the tree regression is _PERFECT_ ... but that's probably not a good thing

## Fine tuning the model - cross validations

### linear v. regression tree v. random forest

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                        scoring="neg_mean_squared_error", cv=10)

tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(tree_rmse_scores)

#### TREE off by $71K (when cross validated - so was an overfit)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                        scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

#### LINEAR off by $69K

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)

forest_mae = mean_absolute_error(housing_labels, housing_predictions)

forest_rmse, forest_mae

#### FOREST seems to overfit, but not as bad as TREE

In [ ]:
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                            scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-forest_scores)).describe()

In [ ]:
display_scores(np.sqrt(-forest_scores))

#### FOREST off by $50K

In [ ]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(housing_prepared, housing_labels)

housing_predictions = svm_reg.predict(housing_prepared)
svm_mse = mean_squared_error(housing_predictions, housing_labels)
svm_rmse = np.sqrt(svm_mse)
svm_rmse


#### Support Vector off by $111K

### Grid search Cross Validation (CV) !!!!
Combo of:
* estimators
* features
* whether or not to bootstrap

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3, 10, 30, 50], 'max_features': [2,4,6,8,10]}
,
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2,3,4]}
]

forest_reg = RandomForestRegressor(random_state=42)

Train for 5 folds:

In [ ]:
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(
    cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

**BEST SCORE**: $49.3 K off <--- (8, 50)

In [ ]:
pd.DataFrame(cvres)

#### Randomized search:

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    'n_estimators': randint(low=1, high=200),
    'max_features': randint(low=1, high=10)
}
forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error',
                               random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), "<----", params)

In [ ]:
print("BEST:", np.sqrt(-rnd_search.best_score_), "<---", rnd_search.best_params_)

**FEATURE IMPORTANCES**

In [ ]:
feature_imps = grid_search.best_estimator_.feature_importances_
feature_imps

In [ ]:
print("SHAME ON YOU MR. GUERON: SO POORLY CODED!!!!")
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
#cat_encoder = cat_pipeline.named_steps["cat_encoder"] # old solution
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_imps, attributes), reverse=True)

#### READY TO EVALUATE ON THE TEST SET!!!!

In [ ]:
final_model = rnd_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
Y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_rmse = np.sqrt(mean_squared_error(final_predictions, Y_test))
final_rmse

95% Confidence Interval:

In [ ]:
from scipy import stats

# stats.sem - computes "standard error of the mean"

confidence = 0.95
squared_errors = (final_predictions - Y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))

In [ ]:
#play around with confidence interval on uniform distribution

r = np.random.uniform(size=100)
m = np.mean(r)
print("mean = ", m)

c = 0.99
stats.t.interval(c, len(r) - 1, loc=m, scale=stats.sem(r))

#### Manual computation of confidence interval of true error (t-scores approach)

In [ ]:
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

#### Manual computation of confidence interval of true error (z-scores approach)

In [ ]:
zscore = stats.norm.ppf((1 + confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - zmargin), np.sqrt(mean + zmargin)